In [ ]:
from collections import defaultdict
import logging
logging.basicConfig(level=logging.DEBUG, format="%(asctime)s: (%(lineno)s) %(levelname)s %(message)s")
import os
import pickle
import re

from gensim.models import KeyedVectors
import json
import matplotlib.pyplot as plt
%pylab inline
pylab.rcParams['figure.figsize'] = (18, 6)
import networkx as nx
import numpy as np
import pprint

from semeval_fca import find_paths_to_object

# Baseline

## Dense embedding

In [ ]:
data_dir ='/mnt/store/hlt/Work/hypernym18-SemEval/'

In [ ]:
embed = KeyedVectors.load_word2vec_format(os.path.join(data_dir, 'mx/1A_UMBC_tokenized.txt_100_sg.vec'))

2018-01-23 19:29:09,375: (204) INFO loading projection weights from /mnt/store/hlt/Work/hypernym18-SemEval/mx/1A_UMBC_tokenized.txt_100_sg.vec


In [ ]:
embed.sav

## Sparse embedding, TODO

In [18]:
# Reading mx
mx_filen = os.path.join(
    data_dir, 
    'mx/1A_UMBC_tokenized.txt_100_sg.vec.gz_True_1000_0.2_unit_True_vocabulary_filtered.alph.reduced2') 
with open(mx_filen, 'rb') as f:
    mx = pickle.load(f, encoding='latin1') 
logging.info(mx.shape)
vocab = [line.strip() for line in open('/home/makrai/repo/fca_hypernymy/data/1A.vocab')]
mx = mx.T
mx = mx[:10000] # TODO

2018-01-23 19:15:29,752: (9) INFO (1000, 164398)


(10000, 1000)

In [20]:
mx.shape

(10000, 1000)

In [22]:
numpy.linalg.norm(mx, axis=0, ord=None)

AxisError: axis 0 is out of bounds for array of dimension 0

In [2]:
logging.info('Computing symilarities...')
sym_mx = mx.T.dot(mx)

2018-01-23 19:00:50,882: (9) INFO (1000, 164398)


In [5]:
sym_mx.getnnz()/sym_mx.shape[0]/sym_mx.shape[1]

0.1349919574154201

In [14]:
logging.info('Computing
sym_mx = sym_mx.astype('float16')

KeyboardInterrupt: 

In [9]:
row = sym_mx[vocab.index('cowpox')]

In [13]:
row.nnz

20932

In [ ]:
neighbors = np.argsort(sym_mx.todense()

In [ ]:
neighbors = np.argsort(sym_mx)

In [ ]:
type(sym_mx)

In [ ]:
neighbors.size()

In [ ]:
neighbors[vocab.index('cowpox')]

In [ ]:
embed.index2word = vocab

In [ ]:
embed.syn0 = mx

In [ ]:
embed['dog']

In [ ]:
'dog' in vocab

In [ ]:
'/mnt/store/hlt/Work/hypernym18-SemEval/mx/1A_UMBC_tokenized.txt_100_sg.vec.gz_True_1000_0.2_unit_True_vocabulary_filtered.alph.reduced'

In [ ]:
with open('{}.w2v'.format(mx_filen), mode='w') as out_file:
    for w, row in zip(vocab, mx.todense()):
        out_file.write('{}\t{}\n'.format(w, ' '.join('{}'.format(coord) for coord in row.tolist()[0])))

# Freq cutoff

In [ ]:
task_dir = '/mnt/permanent/Language/English/Data/SemEval/2018/Hypernym/'

In [ ]:
freq_filen = os.path.join(task_dir, 'frequency_lists/1A_english_frequencylist.txt')
freqs = defaultdict(lambda: 1)
freqs.update({l.split("\t")[0]: int(l.split("\t")[1]) for l in open(freq_filen)})

In [ ]:
golds = [t for l in open(os.path.join(task_dir, 'main_data/training/gold/1A.english.training.gold.txt'))
         for t in l.strip().split('\t')]
plt.hist([0 if g not in freqs else np.log10(freqs[g]) for g in golds])
plt.show()

In [ ]:
vocab = set([l.strip() for l in open(os.path.join(task_dir, 'main_data/vocabulary/1A.english.vocabulary.txt'))])

words_left = lambda x, threshold, vocabulary: [word for word, f in x.items() if f > threshold and word in vocabulary]



for threshold in 25 * np.arange(5):
    s = set(words_left(freqs, threshold, vocab))
    print('{}\t{:.2%}\t{}/{}\t{:.2%}\t{}/{}'.format(threshold, 
                                            len(s)/len(vocab), 
                                            len(s), len(vocab), 
                                             len([g for g in golds if g in s])/len(golds),
                                            len([g for g in golds if g in s]), len(golds)
                                           ))
    

# Tree exploration

In [ ]:
logging.inf('')
J = json.load(open(os.path.join(
    data_dir,
    'concept_tree/1A_UMBC_tokenized.txt_100_sg.vec.gz_True_1000_0.2_unit_True_vocabulary_filtered_reduced.cxt.json'))
logging.inf('')

mind3 visszaadott lista olyan hosszú lesz, mint ahányféleképp el lehetett jutni a gyökérből egy olyan csúcsig, amiben megtalálható volt a lekérdezett szó

* a `path` ezeknek az utaknak a visszafejtését tartalmazza, 
* a `nodes` magát azt a csúcsot, ahol az i. út véget ért
* a `depths` pedig azt hogy a gyökértől milyen mélyen helyezkedik el ez a bizonyos csúcs


elvileg redundáns is, tehát a `paths` i. elemében egy olyan listát kell találj, ami egy gyökérben végződő utat kódol, és a `depths` i. értékéül épp ennek az útnak a hosszát kell lásd

In [ ]:
def freq_based_print(words, max_len=6):
    items = sorted([(w, freqs[w]) for w in words], key=lambda it: it[1], reverse=True)
    ans = '  '.join('{} {:.2}'.format(
        w,#re.sub(' ', '_', w),
        log10(f),) for w, f in items[:max_len])
    len_ = len(words)
    if len_ > max_len:
        ans += '... ({})'.format(len_)
    return ans

In [ ]:
def print_path(query_word):#='dhow'
    """
    Prints pathes from the deepest (most specific) occurrence to the full set.
    """
    paths, nodes, depths = find_paths_to_object(J, query_word)
    logging.info(len(paths))
    max_depth = -1
    for path, node, depth in reversed(list(zip(paths, nodes, depths))):
        if depth < max_depth:
            break
        else:
            max_depth = depth
        print(
            #'dhow' in node, # == True 
            freq_based_print(node), depth)
        for node2 in path[:-1]:
            if len(node2['own_objects']):
                #print('{}\t{}\t{}'.format(my_print(node2['attributes']), node2['Node'], my_print(node2['own_objects'])))
                print('{}'.format(freq_based_print(node2['own_objects'])))#, max_len=11)))
            else:
                atts = 'attributes'
                print('{}: {} (no own object)'.format(atts, node2[atts]))
        print()


In [ ]:
print_path('cowpox')

# Freq of hyper vs hypo

In [ ]:
color_d = {
    (True, True): 'k',
    (True, False): 'b',
    (False, True): 'c',
    (False, False): 'r',
}


query_freqs, gold_freqs, colors, sizes = [], [], [], []
with open(os.path.join(task_dir, 'main_data/training/train.1A.by_freq')) as infile:
    for line in infile:
        query, freq, ent_conc, golds = line.strip().split('\t', maxsplit=3)
        for gold in golds.split('\t'):
            query_freqs.append(freqs[query])
            gold_freqs.append(freqs[gold])
            colors.append(color_d[' ' in query, ' ' in gold])
            sizes.append(10 if ' ' in query else 5)
          
            
plt.xscale('log')
plt.yscale('log')
plt.scatter(query_freqs, gold_freqs, c=colors, s=sizes)

In [ ]:
depths[0]

# Lattice exploration

In [ ]:
dag = nx.read_gpickle('/mnt/permanent/home/makrai/project/hypernym18-SemEval/top500words.gpickle')

In [ ]:
dag.number_of_nodes(), dag.number_of_edges()

In [ ]:
print(dag.nodes['node175']['label'])